<a href="https://colab.research.google.com/github/vishalkumarlondon/data-viz-coursework-group-2/blob/master/colab-notebooks/google_places_api_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Necessary Library

In [0]:
!pip install pygsheets
!pip install --upgrade -q pygsheets
!pip install clean-text[gpl]

In [0]:
!pip install geopandas

In [0]:
!pip install python-sld

#### Authentification Google

In [0]:
# Authentification with google drive and colab
from google.colab import auth
auth.authenticate_user()

# Read google sheet
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
project_id = '' #add your Google Cloud Project ID
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



### Data Preparation

In [0]:
# import the necessary library
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np 
import os
import ast

from datetime import datetime
import re
import string
from pprint import pprint
from cleantext import clean

import seaborn as sns
import matplotlib.pyplot as plt

import pygsheets

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Step0 - Get London, New York and Paris boundary data for boroughs and neighbourhoods

- Data source: http://download.geofabrik.de/europe/great-britain/england/greater-london.html



In [0]:
# London

!wget https://data.london.gov.uk/download/statistical-gis-boundary-files-london/9ba8c833-6370-4b11-abdc-314aa020d5e0/statistical-gis-boundaries-london.zip
!unzip statistical-gis-boundaries-london.zip

In [0]:
# New York

!wget https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nynta_20a.zip
!unzip nynta_20a.zip

#### open shapefile in geopandas dataframe

In [0]:
london_path = '/content/statistical-gis-boundaries-london/ESRI/London_Ward.shp'
london = gpd.read_file(london_path)

newyork_path = '/content/nynta_20a/nynta.shp'
newyork = gpd.read_file(newyork_path)

paris = gpd.read_file("https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=shp&timezone=Europe/London&lang=fr")

london = london.to_crs('EPSG:4326')
newyork = newyork.to_crs('EPSG:4326')
paris = paris.to_crs('EPSG:4326')

In [0]:
london.head(2)

,NAME,GSS_CODE,HECTARES,NONLD_AREA,LB_GSS_CD,BOROUGH,POLY_ID,geometry
0,Chessington South,E05000405,755.173,0.0,E09000021,Kingston upon Thames,50840,"POLYGON ((-0.33068 51.32901, -0.33059 51.32909..."
1,Tolworth and Hook Rise,E05000414,259.464,0.0,E09000021,Kingston upon Thames,117160,"POLYGON ((-0.30846 51.37586, -0.30834 51.37606..."


In [0]:
newyork.head(2)

,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK88,Borough Park,39247.227831,5.400502e+07,"POLYGON ((-73.97605 40.63128, -73.97717 40.630..."
1,4,Queens,081,QN51,Murray Hill,33266.904797,5.248828e+07,"POLYGON ((-73.80379 40.77561, -73.80099 40.775..."


In [0]:
paris.head(2)

,n_sq_ar,c_ar,c_arinsee,l_ar,l_aroff,n_sq_co,surface,perimetre,objectid,longueur,geometry
0,750000001.0,1.0,75101.0,1er Ardt,Louvre,750001537.0,1.824613e+06,6054.936862,1.0,6054.680862,"POLYGON ((2.32801 48.86992, 2.32997 48.86851, ..."
1,750000002.0,2.0,75102.0,2ème Ardt,Bourse,750001537.0,9.911537e+05,4554.104360,2.0,4553.938764,"POLYGON ((2.35152 48.86443, 2.35095 48.86341, ..."


#### limit shapefile to inner areas

In [0]:
#limit London to Inner London Boroughs
INNER_LONDON = ['Camden', 
                'Greenwich', 
                'Hackney', 
                'Hammersmith and Fulham', 
                'Islington',
                'Kensington and Chelsea',
                'Lambeth',
                'Lewisham',
                'Southwark',
                'Tower Hamlets',
                'Wandsworth',
                'Westminster']

print(london.shape)

london = london[london['BOROUGH'].isin(INNER_LONDON)]

print(london.shape)

(649, 8)
(221, 8)


In [0]:
print(newyork.BoroName.value_counts())
print(newyork.NTAName.nunique())
print(newyork.shape)

#limit New York to Manhattan
# newyork = newyork[newyork.BoroName == 'Manhattan']
# print(newyork.shape)

Queens           58
Brooklyn         51
Bronx            38
Manhattan        29
Staten Island    19
Name: BoroName, dtype: int64
195
(195, 8)


#### expand Paris neighbourhoods

In [0]:
'''
Generate five random points inside of each of the 20 boroughs of Paris to make 100 points
similar to London and New York

# https://gis.stackexchange.com/questions/294394/randomly-sample-from-geopandas-dataframe-in-python

'''

from shapely.geometry import Point
import geopandas as gpd
import random

######
def random_points_in_polygon(number, polygon):
    points = []
    min_x, min_y, max_x, max_y = polygon.bounds
    i= 0
    while i < number:
        point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(point):
            points.append(point)
            i += 1
    return points  # returns list of shapely point
######

def Get_RandomPoints_df(dataframe):

  allindex_list = []
  alllatitude_list = []
  alllongitude_list = []

  for row in range(len(paris)):  
      # get inside the api reponse
      points = random_points_in_polygon(5, paris.geometry[row])

      #properties
      index_list = []
      latitude_list = []
      longitude_list = []

      for j, point in enumerate(points):
        # get the index from the dataframe - merge, join for later
         data_index = row
         index_list.append(data_index)

         try:
           latitude = np.around(point.x, decimals=3)
           latitude_list.append(latitude)

           longitude = np.around(point.y, decimals=3)
           longitude_list.append(longitude)

         except:
           latitude = 'NaN'
           latitude_list.append(latitude)

           longitude = 'NaN'
           longitude_list.append(longitude)

      #append all result to final list
      allindex_list.append(index_list)
      alllatitude_list.append(latitude_list)
      alllongitude_list.append(longitude_list)

  dict_df = {'data_index':allindex_list,
             'lat':alllatitude_list,
             'long':alllongitude_list
             }

  apirep_df = pd.DataFrame(dict_df)
  return apirep_df

In [0]:
paris_points = Get_RandomPoints_df(paris)

In [0]:
print(paris.shape, paris_points.shape)

(20, 11) (20, 3)


In [0]:
#make a new dataframe
paris_expanded = pd.concat([paris, paris_points], axis=1)

In [0]:
#drop all duplicate rows by the neighbourhood column in the Ward dataframe
paris_expanded = paris_expanded.drop_duplicates(subset='l_ar')

In [0]:
#limit the new dataframe by the neighbourhood index, lat and long coordinates
paris_expanded = paris_expanded.loc[:, ['l_ar', 'lat', 'long']]

In [0]:
paris_expanded

,l_ar,lat,long
0,1er Ardt,"[2.339, 2.338, 2.342, 2.34, 2.333]","[48.861, 48.862, 48.858, 48.859, 48.864]"
1,2ème Ardt,"[2.346, 2.329, 2.345, 2.341, 2.351]","[48.867, 48.87, 48.87, 48.871, 48.864]"
2,17ème Ardt,"[2.318, 2.318, 2.291, 2.311, 2.325]","[48.896, 48.891, 48.878, 48.89, 48.892]"
3,14ème Ardt,"[2.333, 2.326, 2.336, 2.338, 2.331]","[48.82, 48.833, 48.839, 48.822, 48.835]"
4,20ème Ardt,"[2.412, 2.405, 2.406, 2.415, 2.401]","[48.87, 48.865, 48.854, 48.855, 48.871]"
5,7ème Ardt,"[2.32, 2.315, 2.299, 2.296, 2.312]","[48.856, 48.857, 48.86, 48.86, 48.86]"
6,11ème Ardt,"[2.398, 2.384, 2.369, 2.389, 2.385]","[48.85, 48.864, 48.859, 48.854, 48.86]"
7,13ème Ardt,"[2.346, 2.345, 2.344, 2.361, 2.356]","[48.823, 48.82, 48.817, 48.83, 48.818]"
8,4ème Ardt,"[2.345, 2.351, 2.355, 2.353, 2.348]","[48.854, 48.854, 48.858, 48.858, 48.857]"
9,8ème Ardt,"[2.306, 2.316, 2.32, 2.309, 2.309]","[48.87, 48.87, 48.871, 48.873, 48.878]"


In [0]:
# Explode the multiple list-like columns for each neighbourhood
# https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe

p = (paris_expanded.set_index('l_ar')
   .apply(lambda x: x.apply(pd.Series).stack())
   .reset_index()
   .drop('level_1', 1))

In [0]:
p.shape

(100, 3)

In [0]:
paris = paris.merge(p, on='l_ar', how='inner')

In [0]:
print(paris.shape)
paris.head(1)

(100, 13)


,n_sq_ar,c_ar,c_arinsee,l_ar,l_aroff,n_sq_co,surface,perimetre,objectid,longueur,geometry,lat,long
0,750000001.0,1.0,75101.0,1er Ardt,Louvre,750001537.0,1.824613e+06,6054.936862,1.0,6054.680862,"POLYGON ((2.32801 48.86992, 2.32997 48.86851, ...",2.339,48.861


#### join & create cendriods LDN, NYC, PAR

In [0]:
'''
start with London and New York because we already have 
five random points inside the centre of Paris Boroughs

'''

london = london.loc[:, ['BOROUGH', 'NAME', 'geometry']]
newyork = newyork.loc[:, ['BoroName', 'NTAName', 'geometry']]

london.columns = ['Borough',	'Neighborhood', 'geometry']
newyork.columns = ['Borough',	'Neighborhood', 'geometry']

london['city'] = 'London'
newyork['city'] = 'New York'

gdf_cities = pd.concat([london, newyork])

In [0]:
gdf_cities.head()

,Borough,Neighborhood,geometry,city
208,Lambeth,Streatham South,"POLYGON ((-0.13785 51.42111, -0.13758 51.42057...",London
209,Lambeth,St. Leonard's,"POLYGON ((-0.13750 51.44015, -0.13744 51.44014...",London
210,Lambeth,Streatham Wells,"POLYGON ((-0.11461 51.42347, -0.11488 51.42353...",London
211,Lambeth,Knight's Hill,"POLYGON ((-0.09691 51.42267, -0.09724 51.42263...",London
212,Lambeth,Thornton,"POLYGON ((-0.14362 51.44184, -0.14382 51.44230...",London


In [0]:
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = gdf_cities['geometry'].centroid
x, y = [list(t) for t in zip(*map(getXY, centroidseries))]

In [0]:
gdf_cities['Longitude'] = x
gdf_cities['Latitude'] = y

In [0]:
gdf_cities.head()

,Borough,Neighborhood,geometry,city,Longitude,Latitude
208,Lambeth,Streatham South,"POLYGON ((-0.13785 51.42111, -0.13758 51.42057...",London,-0.130382,51.417380
209,Lambeth,St. Leonard's,"POLYGON ((-0.13750 51.44015, -0.13744 51.44014...",London,-0.133502,51.429830
210,Lambeth,Streatham Wells,"POLYGON ((-0.11461 51.42347, -0.11488 51.42353...",London,-0.121877,51.429980
211,Lambeth,Knight's Hill,"POLYGON ((-0.09691 51.42267, -0.09724 51.42263...",London,-0.107899,51.429287
212,Lambeth,Thornton,"POLYGON ((-0.14362 51.44184, -0.14382 51.44230...",London,-0.136736,51.447030


In [0]:
#Join Paris data with London and New York

paris = paris.loc[:, ['l_ar', 'l_aroff', 'geometry', 'lat', 'long']]
paris.columns = ['Borough',	'Neighborhood', 'geometry', 'Longitude', 'Latitude']
paris['city'] = 'Paris'

gdf_cities = pd.concat([paris, gdf_cities])

In [0]:
print(gdf_cities.shape)
gdf_cities.head()

(516, 6)


,Borough,Neighborhood,geometry,Longitude,Latitude,city
0,1er Ardt,Louvre,"POLYGON ((2.32801 48.86992, 2.32997 48.86851, ...",2.339,48.861,Paris
1,1er Ardt,Louvre,"POLYGON ((2.32801 48.86992, 2.32997 48.86851, ...",2.338,48.862,Paris
2,1er Ardt,Louvre,"POLYGON ((2.32801 48.86992, 2.32997 48.86851, ...",2.342,48.858,Paris
3,1er Ardt,Louvre,"POLYGON ((2.32801 48.86992, 2.32997 48.86851, ...",2.340,48.859,Paris
4,1er Ardt,Louvre,"POLYGON ((2.32801 48.86992, 2.32997 48.86851, ...",2.333,48.864,Paris


In [0]:
gdf_cities.city.value_counts()

London      221
New York    195
Paris       100
Name: city, dtype: int64

#### save neighbourhood centroid coordinates

In [0]:
gdf_cities.to_csv('fashion_week_cities_location_search_Q2_2020.csv')
# !gsutil cp "fashion_week_cities_location_search_Q2_2020.csv" gs://public-data-data_viz-2

Copying file://fashion_week_cities_location_search_Q2_2020.csv [Content-Type=text/csv]...
/ [1 files][  6.1 MiB/  6.1 MiB]                                                
Operation completed over 1 objects/6.1 MiB.                                      


### Step 1 - Get Location Data GoogleAPI 

#### GoogleAPI Key - Get Google Adresse

In [0]:
gdf_osm.head(3)

In [0]:
import urllib
import requests
import json
import time
from tqdm import tqdm

# Google Maps API Key
API_key = '' # your API Key here

## Request with JSON output! 
## https://developers.google.com/places/web-service/search

def Get_PlaceRating(row):

    for i in tqdm(range(10)):
      time.sleep(6)
  
    try:
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='+str(row['Latitude'])+','+str(row['Longitude'])+'&radius=500&type=art_gallery,clothing_store,tourist_attraction,night_club,museum&key='+str(API_key)         
        response = (requests.get(url).text)
        response_json = json.loads(response)
        return response_json  
    except Exception as error:
        raise error

#### API Response in Google Bucket

In [0]:
# run the google API
gdf_cities['API_response'] = gdf_cities.apply(Get_PlaceRating, axis=1)

# save the data into google bucket!
gdf_cities.to_csv('new.csv')
!gsutil cp  'new.csv' gs://public-data-data_viz-2/

### Step 2 - Clean OSM API Response

#### Read data 

In [0]:
#pull data from GCP bucket
!gsutil cp gs://public-data-data_viz-2/fw19_placesapi_reponse.csv fw19_placesapi_reponse.csv

Copying gs://public-data-data_viz-2/fw19_placesapi_reponse.csv...
/ [1 files][ 16.0 MiB/ 16.0 MiB]                                                
Operation completed over 1 objects/16.0 MiB.                                     


In [0]:
data = pd.read_csv('fw19_placesapi_reponse.csv')
data = data.drop(columns='Unnamed: 0')

In [0]:
data.head(2)

,Borough,Neighborhood,geometry,Longitude,Latitude,city,API_response
0,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.339,48.861,Paris,"{'html_attributions': [], 'next_page_token': '..."
1,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.338,48.862,Paris,"{'html_attributions': [], 'next_page_token': '..."


#### Get the Ratings (Parse Json API_response)

###### step1 - get non-empty api_response

In [0]:
def Get_EmptyReponse(dataframe):
  
  api_replist = []

  for row in range(len(dataframe)): 
          # get inside the api reponse
          api_rep = ast.literal_eval(dataframe['API_response'][row])
          # check if list is empty? if yes - nan, if no - api_rep
          if len(api_rep['place_id']) == 0:
            api_rep = 'NaN'
          else:
            api_rep = api_rep['place_id']

          api_replist.append(api_rep)

  dataframe['api_rep'] = api_replist
  return dataframe

In [0]:
# data = Get_EmptyReponse(data)
# data.head(1)

# data_notempty = data[data['api_rep'] != 'NaN']
# print('Length of data: '+str(len(data)))
# print('Length of data with an API reponse: '+str(len(data_notempty)))

###### step2 - access api_response data

In [0]:
import pprint

def Get_APIresponse_df(dataframe):

  allindex_list = []
  allname_list = []
  alladdress_list = []
  alllatitude_list = []
  alllongitude_list = []
  allrating_list = []
  alluser_ratings_list = []
  alltypes_list = []

  apirep_list = []
  apiisrep_list = []

  for row in range(len(dataframe)):  
      # get inside the api reponse
      api_rep = ast.literal_eval(dataframe['API_response'][row])
      results = api_rep['results']
      if len(results) == 0:
        results_new = 'False'
      else:
        results_new = 'True'

      #properties
      index_list = []
      name_list = []
      address_list = []
      latitude_list = []
      longitude_list = []
      rating_list = []
      user_ratings_list = []
      types_list = []

      for i in range(len(results)):
        # get the index from the dataframe - merge, join for later
         data_index = row
         index_list.append(data_index)
         try:
           name = results[i]['name']
           name_list.append(name)

           address = results[i]['vicinity']
           address_list.append(address)

           latitude = results[i]['geometry']['location']['lat']
           latitude_list.append(latitude)

           longitude = results[i]['geometry']['location']['lng']
           longitude_list.append(longitude)

           rating = results[i]['rating']
           rating_list.append(rating)

           user_ratings = results[i]['user_ratings_total']
           user_ratings_list.append(user_ratings)

           types = results[i]['types']
           types_list.append(types)

         except:
           name = 'NaN'
           name_list.append(name)

           address = 'NaN'
           address_list.append(address)

           latitude = 'NaN'
           latitude_list.append(latitude)

           longitude = 'NaN'
           longitude_list.append(longitude)

           rating = 'NaN'
           rating_list.append(rating)

           user_ratings = 'NaN'
           user_ratings_list.append(user_ratings)

      #append all result to final list
      allindex_list.append(index_list)
      allname_list.append(name_list)
      alladdress_list.append(address_list)
      alllatitude_list.append(latitude_list)
      alllongitude_list.append(longitude_list)
      allrating_list.append(rating_list)
      alluser_ratings_list.append(user_ratings_list)
      alltypes_list.append(types_list)
      
      apiisrep_list.append(results_new)
      apirep_list.append(results)

  dict_df = {'data_index':allindex_list,
             'name':allname_list,
             'address':alladdress_list,
             'lat':alllatitude_list,
             'long':alllongitude_list,
             'rating':allrating_list,
             'user_ratings_total': alluser_ratings_list,
             'types': alltypes_list,

             'is_apirep':apiisrep_list,
             'api_rep':apirep_list}

  apirep_df = pd.DataFrame(dict_df)
  return apirep_df

In [0]:
apirep_df = Get_APIresponse_df(data)

In [0]:
apirep_df.head(2)

,data_index,name,address,lat,long,rating,user_ratings_total,types,is_apirep,api_rep
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Paris, NaN, Hôtel Ducs de Bourgogne, Hotel Re...","[Paris, NaN, 19 Rue du Pont Neuf, Paris, 19 Ru...","[48.85661400000001, NaN, 48.86034799999999, 48...","[2.3522219, NaN, 2.344054, 2.341004799999999, ...","[NaN, 4.5, 4.6, 4.2, 4.5, 4, 3.8, 3.6, 4.5, 4....","[NaN, 264, 104, 329, 111, 233, 168, 324, 282, ...","[[lodging, point_of_interest, establishment], ...",True,[{'geometry': {'location': {'lat': 48.85661400...
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Paris, NaN, Hotel Regina Louvre, Hôtel Ducs d...","[Paris, NaN, 2 Place des Pyramides, Paris, 19 ...","[48.85661400000001, NaN, 48.86386539999999, 48...","[2.3522219, NaN, 2.3325488, 2.344054, 2.342582...","[NaN, 4.5, 4.5, 4.2, 4.6, 3.2, 4.5, 3.3, 4, 4....","[NaN, 493, 264, 329, 104, 215, 111, 892, 233, ...","[[lodging, bar, restaurant, food, point_of_int...",True,[{'geometry': {'location': {'lat': 48.85661400...


In [0]:
apirep_df = apirep_df.iloc[:,:8]

In [0]:
print(data.shape, apirep_df.shape)

(516, 7) (516, 8)


###### step3 - join api_response data on index

In [0]:
#make a new dataframe by concatinating the Ward dataframe and the API repsonse dataframe on the same index
new_data = pd.concat([data, apirep_df], axis=1)

In [0]:
new_data.head(2)

,Neighborhood,name,address,rating,user_ratings_total,types,lat,long
0,Louvre,"[Paris, NaN, Hôtel Ducs de Bourgogne, Hotel Re...","[Paris, NaN, 19 Rue du Pont Neuf, Paris, 19 Ru...","[NaN, 4.5, 4.6, 4.2, 4.5, 4, 3.8, 3.6, 4.5, 4....","[NaN, 264, 104, 329, 111, 233, 168, 324, 282, ...","[[lodging, point_of_interest, establishment], ...","[48.85661400000001, NaN, 48.86034799999999, 48...","[2.3522219, NaN, 2.344054, 2.341004799999999, ..."
1,Bourse,"[Paris, NaN, Hotel Royal Aboukir, Hotel Des Bo...","[Paris, NaN, 106 Rue d'Aboukir, Paris, 10 Rue ...","[NaN, 3, 4.5, 3.6, 2.6, 3.8, 3.7, 3.4, 3.4, 4,...","[NaN, 262, 23, 236, 202, 141, 118, 90, 176, 13...","[[lodging, point_of_interest, establishment], ...","[48.85661400000001, NaN, 48.86856479999999, 48...","[2.3522219, NaN, 2.349895800000001, 2.3486459,..."


In [0]:
#drop all duplicate rows by the NAME column in the Ward dataframe
new_data = new_data.drop_duplicates(subset='Neighborhood')

In [0]:
new_data.head(2)

,Borough,Neighborhood,geometry,Longitude,Latitude,city,API_response,data_index,name,address,lat,long,rating,user_ratings_total,types
0,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.339,48.861,Paris,"{'html_attributions': [], 'next_page_token': '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Paris, NaN, Hôtel Ducs de Bourgogne, Hotel Re...","[Paris, NaN, 19 Rue du Pont Neuf, Paris, 19 Ru...","[48.85661400000001, NaN, 48.86034799999999, 48...","[2.3522219, NaN, 2.344054, 2.341004799999999, ...","[NaN, 4.5, 4.6, 4.2, 4.5, 4, 3.8, 3.6, 4.5, 4....","[NaN, 264, 104, 329, 111, 233, 168, 324, 282, ...","[[lodging, point_of_interest, establishment], ..."
5,2ème Ardt,Bourse,"POLYGON ((2.351518483670822 48.8644258050741, ...",2.346,48.867,Paris,"{'html_attributions': [], 'next_page_token': '...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[Paris, NaN, Hotel Royal Aboukir, Hotel Des Bo...","[Paris, NaN, 106 Rue d'Aboukir, Paris, 10 Rue ...","[48.85661400000001, NaN, 48.86856479999999, 48...","[2.3522219, NaN, 2.349895800000001, 2.3486459,...","[NaN, 3, 4.5, 3.6, 2.6, 3.8, 3.7, 3.4, 3.4, 4,...","[NaN, 262, 23, 236, 202, 141, 118, 90, 176, 13...","[[lodging, point_of_interest, establishment], ..."


In [0]:
new_data.drop('API_response', axis=1, inplace=True)
new_data.reset_index(inplace=True)

In [0]:
new_data.columns

Index(['index', 'Borough', 'Neighborhood', 'geometry', 'Longitude', 'Latitude',
       'city', 'data_index', 'name', 'address', 'lat', 'long', 'rating',
       'user_ratings_total', 'types'],
      dtype='object')

In [0]:
new_data = new_data.loc[:, ['Neighborhood', 'name',
                            'address', 'rating', 
                            'user_ratings_total', 'types', 
                            'lat', 'long',]]

In [0]:
# new_data.reset_index(inplace=True)

In [0]:
new_data.head()

,Neighborhood,name,address,rating,user_ratings_total,types,lat,long
0,Louvre,"[Paris, NaN, Hôtel Ducs de Bourgogne, Hotel Re...","[Paris, NaN, 19 Rue du Pont Neuf, Paris, 19 Ru...","[NaN, 4.5, 4.6, 4.2, 4.5, 4, 3.8, 3.6, 4.5, 4....","[NaN, 264, 104, 329, 111, 233, 168, 324, 282, ...","[[lodging, point_of_interest, establishment], ...","[48.85661400000001, NaN, 48.86034799999999, 48...","[2.3522219, NaN, 2.344054, 2.341004799999999, ..."
1,Bourse,"[Paris, NaN, Hotel Royal Aboukir, Hotel Des Bo...","[Paris, NaN, 106 Rue d'Aboukir, Paris, 10 Rue ...","[NaN, 3, 4.5, 3.6, 2.6, 3.8, 3.7, 3.4, 3.4, 4,...","[NaN, 262, 23, 236, 202, 141, 118, 90, 176, 13...","[[lodging, point_of_interest, establishment], ...","[48.85661400000001, NaN, 48.86856479999999, 48...","[2.3522219, NaN, 2.349895800000001, 2.3486459,..."
2,Batignolles-Monceau,"[Paris, NaN, Hôtel De Nantes, Art Hôtel Congrè...","[Paris, NaN, 3 Rue Boulay, Paris, 44 Rue Gauth...","[NaN, 2.7, 3.9, 2.6, 4.2, NaN, 3.8, 4.1, 3.3, ...","[NaN, 241, 275, 116, 66, NaN, 96, 23, 41, 368,...","[[lodging, point_of_interest, establishment], ...","[48.85661400000001, NaN, 48.89300189999999, 48...","[2.3522219, NaN, 2.3167206, 2.321625, 2.317733..."
3,Observatoire,"[Paris, NaN, Hôtel du Parc Montsouris, ibis bu...","[Paris, NaN, 4 Rue du Parc de Montsouris, Pari...","[NaN, 3.6, 3.6, 3.8, 3.9, 4.4, 4.3, 4.8, 4.5, ...","[NaN, 177, 1244, 298, 360, 2659, 7, 11, 453, 2...","[[lodging, point_of_interest, establishment], ...","[48.85661400000001, NaN, 48.8224155, 48.819484...","[2.3522219, NaN, 2.3345877, 2.3263139, 2.33069..."
4,Ménilmontant,"[Paris, NaN, Crêche Sainte Amélie, Bonjour Par...","[Paris, NaN, 7 Rue de Noisy-le-Sec, Paris, 21 ...","[NaN, 5, 5, 4.9, 4, 4.6, 3.4, 5, 2.4, 5, 5, 3....","[NaN, 1, 2, 10, 20, 69, 110, 1, 10, 1, 2, 19, ...","[[point_of_interest, establishment], [lodging,...","[48.85661400000001, NaN, 48.8719349, 48.866505...","[2.3522219, NaN, 2.4114796, 2.4079531, 2.41511..."


In [0]:
new_data[new_data.index.duplicated()]

,Neighborhood,name,address,rating,user_ratings_total,types,lat,long


###### step4 - explode list-like columns by Neighbourhood

In [0]:
# How to explode multiple list-like columns
# https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe

x = (new_data.set_index('Neighborhood')
   .apply(lambda x: x.apply(pd.Series).stack())
   .reset_index()
   .drop('level_1', 1))

In [0]:
x.sample(20)

,Neighborhood,name,address,rating,user_ratings_total,types,lat,long
7004,Northcote,Palmer,"125B, 125C Northcote Road, London",4.3,527,"[real_estate_agency, finance, general_contract...",51.4563,-0.165187
5887,Livesey,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8188,Rego Park,AT&T,"96-14 Queens Boulevard, Rego Park",4.3,572,"[pharmacy, drugstore, convenience_store, food,...",40.7296,-73.8621
1018,Bow West,Scootech,"425-427 Roman Road, London",3.8,36,"[cafe, bakery, restaurant, food, point_of_inte...",51.5318,-0.0361959
6155,Mariner's Harbor-Arlington-Port Ivory-Granitev...,American Rolling Door Ltd,"40 Dolson Place, Staten Island",NaN,NaN,NaN,40.629,-74.1698
333,Baisley Park,"Stealth Productions, LLC","153-28 122nd Avenue, Jamaica",NaN,NaN,NaN,40.677,-73.7874
1677,Cantelowes,York Way Spacious Apartments,"69 York Way, London",4.9,50,"[gas_station, point_of_interest, establishment]",51.5448,-0.126479
8191,Rego Park,Subway,"95-58 Queens Boulevard, Queens",4.1,21,"[bar, night_club, point_of_interest, establish...",40.73,-73.8629
419,Batignolles-Monceau,TOTAL,"190 Avenue de Clichy, Paris",4.5,54,"[bank, atm, finance, point_of_interest, establ...",48.8936,2.31526
4267,Gravesend,Horizon Christian Fellowship,"2585 West 13th Street, Brooklyn",3.5,202,"[school, point_of_interest, establishment]",40.5851,-73.9816


In [0]:
x.shape

(11411, 8)

###### step5 - clean data and push to GCP

In [0]:
x.lat.isna().sum()

159

In [0]:
#remove data when latitude is NaN
x = x[x.lat !='NaN']

In [0]:
x.sample(20)

,Neighborhood,name,address,rating,user_ratings_total,types,lat,long
10363,Village,Dulwich Vintners,"85-87 Dulwich Village, London",NaN,NaN,"[health, point_of_interest, establishment]",51.4491,-0.085066
4966,Homecrest,"Ezphonerepairs of brooklyn , ny","2020 East 7th Street, Brooklyn",NaN,NaN,NaN,40.6013,-73.965
1126,Brockley,Saxophone tutor,"21 Wickham Road, London",NaN,NaN,"[park, point_of_interest, establishment]",51.4677,-0.0291841
10722,West New Brighton-New Brighton-St. George,"Dr. Vittorio E. Rotella Jr, MD","355 Bard Avenue, Staten Island",NaN,NaN,NaN,40.6357,-74.106
213,Askew,Sufi Restaurant,"70 Askew Road, London",4.2,21,"[library, point_of_interest, establishment]",51.5041,-0.243725
10084,Town,National Federation Of Womens Institutes,"104 New King's Road, London",4.6,26,"[point_of_interest, establishment]",51.4723,-0.204246
10069,Town,London,London,NaN,NaN,"[point_of_interest, clothing_store, store, est...",51.5074,-0.127758
9505,Streatham Hill,London,London,NaN,NaN,"[point_of_interest, establishment]",51.5074,-0.127758
1680,Cantelowes,Ben Nevis Clothing,"237 Royal College Street, London",3.9,212,NaN,51.5424,-0.139097
6191,Maspeth,Mount Olivet Cemetery,"6540 Grand Avenue, Maspeth",4.2,13,"[point_of_interest, establishment]",40.723,-73.8993


In [0]:
data.head()

,Borough,Neighborhood,geometry,Longitude,Latitude,city,API_response
0,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.339,48.861,Paris,"{'html_attributions': [], 'next_page_token': '..."
1,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.338,48.862,Paris,"{'html_attributions': [], 'next_page_token': '..."
2,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.342,48.858,Paris,"{'html_attributions': [], 'next_page_token': '..."
3,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.340,48.859,Paris,"{'html_attributions': [], 'next_page_token': '..."
4,1er Ardt,Louvre,"POLYGON ((2.328007329038849 48.86991742140716,...",2.333,48.864,Paris,"{'html_attributions': [], 'next_page_token': '..."


In [0]:
city_data = data.loc[:, ['Neighborhood', 'city']].merge(x, on='Neighborhood', how='inner')

In [0]:
print(city_data.shape, x.shape)

(10092, 9) (8454, 8)


In [0]:
city_data.head()

,Neighborhood,city,name,address,rating,user_ratings_total,types,lat,long
0,Louvre,Paris,Paris,Paris,NaN,NaN,"[lodging, point_of_interest, establishment]",48.8566,2.35222
1,Louvre,Paris,Hôtel Ducs de Bourgogne,"19 Rue du Pont Neuf, Paris",4.6,104,"[lodging, point_of_interest, establishment]",48.8603,2.34405
2,Louvre,Paris,Hotel Relais Du Louvre,"19 Rue des Prêtres Saint-Germain l'Auxerrois, ...",4.2,329,"[lodging, point_of_interest, establishment]",48.8592,2.341
3,Louvre,Paris,L'Empire Paris,"48 Rue de l'Arbre Sec, Paris",4.5,111,"[lodging, point_of_interest, establishment]",48.8607,2.34258
4,Louvre,Paris,Hôtel 3 étoiles de la Place du Louvre Paris 1e...,"21 Rue des Prêtres Saint-Germain l'Auxerrois, ...",4,233,"[lodging, point_of_interest, establishment]",48.8592,2.3409


In [0]:
city_data.drop_duplicates(subset=['Neighborhood', 'city', 'name'], inplace=True)

print(city_data.shape)

(8456, 9)


In [0]:
# save the data into google bucket!
city_data.to_csv('new.csv')
!gsutil cp  'new.csv' gs://public-data-data_viz-2/

Copying file://fw19_placesapi_reponse_clean.csv [Content-Type=text/csv]...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      
